In [1]:
from pyspark.sql import SparkSession
import os


In [2]:
spark = SparkSession.builder \
        .appName("app") \
        .config("spark.driver.memory", "45g") \
        .config("spark.executor.memory", "45g") \
        .config("spark.jars", "postgresql-42.3.3.jar") \
        .getOrCreate()
#spark.sparkContext.setLogLevel("DEBUG")

23/10/16 07:57:06 WARN Utils: Your hostname, server-rennweg resolves to a loopback address: 127.0.1.1; using 192.168.1.196 instead (on interface enp5s0f1)
23/10/16 07:57:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/16 07:57:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/16 07:57:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/16 07:57:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
spark.sparkContext.setLogLevel("WARN") # ALL, DEBUG, WARN,
#spark.sparkContext.setLogLevel("ALL") # ALL, DEBUG, WARN,

In [4]:
username = os.environ.get('USERNAME', 'snap')
password = os.environ.get('PASSWORD', 'snap')
dbname = os.environ.get('DBNAME', 'snap')
dbhost = os.environ.get('DBHOST', 'localhost')

df_tables = spark.read.format("jdbc") \
    .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "information_schema.tables") \
    .option("user", username) \
    .option("password", password) \
    .load()

for idx, row in df_tables.toPandas().iterrows():
        if row.table_schema == 'public':
            table_name = row.table_name
            df = spark.read.format("jdbc") \
                .option("url", f'jdbc:postgresql://{dbhost}:5432/{dbname}') \
                .option("driver", "org.postgresql.Driver") \
                .option("dbtable", table_name) \
                .option("user", username) \
                .option("password", password) \
                .load()
    
            print(table_name)
            print(df.show())
            df.createOrReplaceTempView(table_name)

patents


+--------+-------+
|fromnode| tonode|
+--------+-------+
| 3858241| 956203|
| 3858241|1324234|
| 3858241|3398406|
| 3858241|3557384|
| 3858241|3634889|
| 3858242|1515701|
| 3858242|3319261|
| 3858242|3668705|
| 3858242|3707004|
| 3858243|2949611|
| 3858243|3146465|
| 3858243|3156927|
| 3858243|3221341|
| 3858243|3574238|
| 3858243|3681785|
| 3858243|3684611|
| 3858244|  14040|
| 3858244|  17445|
| 3858244|2211676|
| 3858244|2635670|
+--------+-------+
only showing top 20 rows

None


In [ ]:
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [5]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()
spark.sql("SET spark.sql.yannakakis.enabled = true").show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...|false|
+--------------------+-----+

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+



In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = false").show()

In [ ]:
spark.sql("SET spark.sql.yannakakis.enabled = true").show()
spark.sql("SET spark.sql.yannakakis.countGroupInLeaves = false").show()

In [ ]:
spark.sql("SET spark.sql.cbo.enabled = true").show()

In [ ]:
spark.sql("ANALYZE TABLE part COMPUTE STATISTICS;").show()

In [6]:
df = spark.sql("""select count(*) from patents p1, patents p2, patents p3 where p1.toNode = p2.fromNode AND p2.toNode = p3.fromNode""")
df.show()

df.explain(True)

23/10/16 07:58:44 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(Etc/UTC)) AS toprettystring(count(1))#83]
+- Project
   +- Join Inner, (toNode#76 = fromNode#77)
      :- Project [tonode#76]
      :  +- Join Inner, (toNode#25 = fromNode#75)
      :     :- Project [tonode#25]
      :     :  +- Filter isnotnull(toNode#25)
      :     :     +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#75) AND isnotnull(toNode#76))
      :        +- Relation [fromnode#75,tonode#76] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#77]
         +- Filter isnotnull(fromNode#77)
            +- Relation [fromnode#77,tonode#78] JDBCRelation(patents) [numPartitions=1]

23/10/16 07:58:44 WARN RewriteJoinsAsSemijoins: agg(project(join))
23/10/16 07:58:44 WARN RewriteJoinsAsSemijoins: items: List(Project [tonode#25]
+- Filter isnotnull(toNode#25)
   +- Relation [fr

+---------+
| count(1)|
+---------+
|361611968|
+---------+

== Parsed Logical Plan ==
'Project [unresolvedalias('count(1), None)]
+- 'Filter (('p1.toNode = 'p2.fromNode) AND ('p2.toNode = 'p3.fromNode))
   +- 'Join Inner
      :- 'Join Inner
      :  :- 'SubqueryAlias p1
      :  :  +- 'UnresolvedRelation [patents], [], false
      :  +- 'SubqueryAlias p2
      :     +- 'UnresolvedRelation [patents], [], false
      +- 'SubqueryAlias p3
         +- 'UnresolvedRelation [patents], [], false

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#80L]
+- Filter ((toNode#25 = fromNode#75) AND (toNode#76 = fromNode#77))
   +- Join Inner
      :- Join Inner
      :  :- SubqueryAlias p1
      :  :  +- SubqueryAlias patents
      :  :     +- View (`patents`, [fromnode#24,tonode#25])
      :  :        +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [numPartitions=1]
      :  +- SubqueryAlias p2
      :     +- SubqueryAlias patents
      :        +- View (`patent

23/10/16 07:58:56 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [count(1) AS count(1)#80L]
+- Project
   +- Join Inner, (toNode#76 = fromNode#77)
      :- Project [tonode#76]
      :  +- Join Inner, (toNode#25 = fromNode#75)
      :     :- Project [tonode#25]
      :     :  +- Filter isnotnull(toNode#25)
      :     :     +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#75) AND isnotnull(toNode#76))
      :        +- Relation [fromnode#75,tonode#76] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#77]
         +- Filter isnotnull(fromNode#77)
            +- Relation [fromnode#77,tonode#78] JDBCRelation(patents) [numPartitions=1]

23/10/16 07:58:56 WARN RewriteJoinsAsSemijoins: agg(project(join))
23/10/16 07:58:56 WARN RewriteJoinsAsSemijoins: items: List(Project [tonode#25]
+- Filter isnotnull(toNode#25)
   +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [nu

In [ ]:
df = spark.sql("""select avg(i_current_price) from item, inventory where i_item_sk = inv_item_sk""")
df = spark.sql("""select avg(ss_quantity) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select avg(ss_list_price) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")
#df = spark.sql("""select count(isnull(i_current_price)) from item, inventory where i_item_sk = inv_item_sk""")
#df = spark.sql("""select sum(if (isnull(ss_quantity), 0, ss_quantity)) from store_sales,store where s_store_sk = ss_store_sk""")

df.show()

df.explain(True)

In [8]:
def run_query(file):
    with open(file, 'r') as f:
        query = '\n'.join(filter(lambda line: not line.startswith('limit') and not line.startswith('-'), f.readlines()))
        
        print("running query: \n" + query)
        return spark.sql(query)

In [ ]:
df = spark.sql("""
SELECT ps_partkey, count(*) from part, partsupp
WHERE p_partkey = ps_partkey
GROUP BY ps_partkey
""")

df.show()

df.explain(True)

In [ ]:
df = spark.sql("SELECT COUNT(p_size) / COUNT(DISTINCT p_size) FROM part")
df.show()
df = spark.sql("SELECT COUNT(p_retailprice) / COUNT(DISTINCT p_retailprice) FROM part")
df.show()

In [ ]:
df_t1 = spark.createDataFrame([(1,1), (2,1), (2,2), (3,2), (3,3), (4,3), (4,3), (5,2), (5,1), (6,4)], schema=("a","b"))
df_t1.createOrReplaceTempView("t1")
df_t2 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("c","d"))
df_t2.createOrReplaceTempView("t2")
df_t3 = spark.createDataFrame([(1,1), (2,1), (3,2), (3,2), (3,3), (3,3), (4,3), (4,2), (5,1), (6,4)], schema=("e","f"))
df_t3.createOrReplaceTempView("t3")

query = "select median(a) from t1, t2 where b = c"
#query = "select percentile(a, 0.5, b) from t1, t2 where b = c"
#query = "select median(a) from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"
#query = "select count(*) from t1, t2 where b = c"
#query = "select *a from t1 where EXISTS (SELECT 1 FROM t2 WHERE b = c)"

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

df = spark.sql(query)
df.show()

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

df = spark.sql(query)
df.show()

In [ ]:
spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "6")
#spark.conf.set("spark.executor.instances", "6")
spark.conf.set("spark.sql.shuffle.partitions", "6")

In [ ]:
import pandas as pd
import time

def benchmark(query):
    df0 = run_query(query)
    df0.show()
    
    spark.sql("SET spark.sql.yannakakis.enabled = true").show()

    start_time = time.time()

    df1 = run_query(query)
    df1.show()
    #df1.explain(True)

    end_time = time.time()
    yannakakis_time = end_time - start_time

    spark.sql("SET spark.sql.yannakakis.enabled = false").show()

    start_time = time.time()

    df2 = run_query(query)
    df2.show()
    #df2.explain(True)

    end_time = time.time()
    ref_time = end_time - start_time
    
    #return [query, ref_time, yannakakis_time]
    return [query, ref_time, yannakakis_time]


queries = ['tpch-kit/dbgen/queries/postgres/2.sql',
           'tpch-kit/dbgen/queries/postgres/11.sql', 
           'tpch-kit/dbgen/queries/postgres/11-hint.sql',
           'median-1.sql',
           'median-2.sql', 
           'median-3.sql', 
           'median-4.sql', 
           'median-5.sql',
            'median-1-hint.sql',
           'median-2-hint.sql', 
           'median-3-hint.sql', 
           'median-4-hint.sql', 
           'median-5-hint.sql']

results = [benchmark(q) for q in queries]

df = pd.DataFrame(results, columns = ['query', 'ref_time', 'yannakakis_time'])

print(df)

df.to_csv("results.csv")
    

#print(f'row count: {df1.count()} vs. {df2.count()}' )
    #print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

In [ ]:
#spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false")
#spark.conf.set("spark.executor.cores", "1")
#spark.conf.set("spark.executor.instances", "1")
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [16]:
## Compare result
import time
#query = 'tpch-kit/dbgen/queries/postgres/2.sql'
#query = 'tpch-kit/dbgen/queries/postgres/13.sql'
#query = 'count-3.sql'
#query = 'tpch-kit/dbgen/queries/postgres/11.sql'
#query = '11-simple.sql'
query = 'median-2-hint.sql'
#query = 'tpch-kit/dbgen/queries/postgres/7.sql'
#query = '13-simple.sql'
query = 'dsb-queries/agg_queries/query101.sql'
query = 'snap-queries/patents02.sql'

spark.sql("SET spark.sql.yannakakis.enabled = true").show()

start_time = time.time()

df1 = run_query(query)
df1.show()
df1.explain(mode="extended")

end_time = time.time()
yannakakis_time = end_time - start_time

spark.sql("SET spark.sql.yannakakis.enabled = false").show()

start_time = time.time()

df2 = run_query(query)
df2.show()
df2.explain(mode="extended")

end_time = time.time()
ref_time = end_time - start_time

#print(f'row count: {df1.count()} vs. {df2.count()}' )
print(f'time ref: {ref_time}\ntime yannakakis: {yannakakis_time}')

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.yannaka...| true|
+--------------------+-----+

running query: 
select count(*) from patents p1, patents p2, patents p3, patents p4 where p1.toNode = p2.fromNode AND p2.toNode = p3.fromNode AND p3.toNode = p4.fromNode



23/10/17 11:55:01 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [toprettystring(count(1), Some(Etc/UTC)) AS toprettystring(count(1))#1064]
+- Project
   +- Join Inner, (toNode#1057 = fromNode#1058)
      :- Project [tonode#1057]
      :  +- Join Inner, (toNode#1055 = fromNode#1056)
      :     :- Project [tonode#1055]
      :     :  +- Join Inner, (toNode#25 = fromNode#1054)
      :     :     :- Project [tonode#25]
      :     :     :  +- Filter isnotnull(toNode#25)
      :     :     :     +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [numPartitions=1]
      :     :     +- Filter (isnotnull(fromNode#1054) AND isnotnull(toNode#1055))
      :     :        +- Relation [fromnode#1054,tonode#1055] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#1056) AND isnotnull(toNode#1057))
      :        +- Relation [fromnode#1056,tonode#1057] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#1058]
         

23/10/17 11:55:14 WARN RewriteJoinsAsSemijoins: applying yannakakis rewriting to join: Aggregate [count(1) AS count(1)#1061L]
+- Project
   +- Join Inner, (toNode#1057 = fromNode#1058)
      :- Project [tonode#1057]
      :  +- Join Inner, (toNode#1055 = fromNode#1056)
      :     :- Project [tonode#1055]
      :     :  +- Join Inner, (toNode#25 = fromNode#1054)
      :     :     :- Project [tonode#25]
      :     :     :  +- Filter isnotnull(toNode#25)
      :     :     :     +- Relation [fromnode#24,tonode#25] JDBCRelation(patents) [numPartitions=1]
      :     :     +- Filter (isnotnull(fromNode#1054) AND isnotnull(toNode#1055))
      :     :        +- Relation [fromnode#1054,tonode#1055] JDBCRelation(patents) [numPartitions=1]
      :     +- Filter (isnotnull(fromNode#1056) AND isnotnull(toNode#1057))
      :        +- Relation [fromnode#1056,tonode#1057] JDBCRelation(patents) [numPartitions=1]
      +- Project [fromnode#1058]
         +- Filter isnotnull(fromNode#1058)
           

+----------+
|  count(1)|
+----------+
|1380939105|
+----------+

== Parsed Logical Plan ==
'Project [unresolvedalias('count(1), None)]
+- 'Filter ((('p1.toNode = 'p2.fromNode) AND ('p2.toNode = 'p3.fromNode)) AND ('p3.toNode = 'p4.fromNode))
   +- 'Join Inner
      :- 'Join Inner
      :  :- 'Join Inner
      :  :  :- 'SubqueryAlias p1
      :  :  :  +- 'UnresolvedRelation [patents], [], false
      :  :  +- 'SubqueryAlias p2
      :  :     +- 'UnresolvedRelation [patents], [], false
      :  +- 'SubqueryAlias p3
      :     +- 'UnresolvedRelation [patents], [], false
      +- 'SubqueryAlias p4
         +- 'UnresolvedRelation [patents], [], false

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#1061L]
+- Filter (((toNode#25 = fromNode#1054) AND (toNode#1055 = fromNode#1056)) AND (toNode#1057 = fromNode#1058))
   +- Join Inner
      :- Join Inner
      :  :- Join Inner
      :  :  :- SubqueryAlias p1
      :  :  :  +- SubqueryAlias patents
      :  :  :    

+----------+
|  count(1)|
+----------+
|1380939105|
+----------+

== Parsed Logical Plan ==
'Project [unresolvedalias('count(1), None)]
+- 'Filter ((('p1.toNode = 'p2.fromNode) AND ('p2.toNode = 'p3.fromNode)) AND ('p3.toNode = 'p4.fromNode))
   +- 'Join Inner
      :- 'Join Inner
      :  :- 'Join Inner
      :  :  :- 'SubqueryAlias p1
      :  :  :  +- 'UnresolvedRelation [patents], [], false
      :  :  +- 'SubqueryAlias p2
      :  :     +- 'UnresolvedRelation [patents], [], false
      :  +- 'SubqueryAlias p3
      :     +- 'UnresolvedRelation [patents], [], false
      +- 'SubqueryAlias p4
         +- 'UnresolvedRelation [patents], [], false

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#1160L]
+- Filter (((toNode#25 = fromNode#1153) AND (toNode#1154 = fromNode#1155)) AND (toNode#1156 = fromNode#1157))
   +- Join Inner
      :- Join Inner
      :  :- Join Inner
      :  :  :- SubqueryAlias p1
      :  :  :  +- SubqueryAlias patents
      :  :  :    